In [1]:
import sys

sys.path.append("../")
import os
import math
import numpy as np
import cv2
import pandas as pd
import tikzplotlib
from pedrec.configs.dataset_configs import get_sim_dataset_cfg_default
from pedrec.datasets.pedrec_dataset import PedRecDataset
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.visualizers.skeleton_3d_visualizer import add_skeleton_3d_to_axes
from pedrec.visualizers.visualization_helper_3d import draw_grid_3d, draw_origin_3d
from random import randint
import matplotlib.pyplot as plt
%matplotlib widget


In [2]:
cfg = PedRecNet50Config()

# ROM Train
dataset_cfg = get_sim_dataset_cfg_default()
dataset_root = "data/datasets/ROMb/"
dataset_df_filename = "rt_rom_01b.pkl"

dataset = PedRecDataset(dataset_root, dataset_df_filename, DatasetType.VALIDATE, dataset_cfg, cfg.model.input_size, None)
dataset_length = len(dataset)

In [3]:
fig, ax = plt.subplots(3,3, figsize=(10,10))
fig_3d = plt.figure(figsize=(10,10))

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        entry = dataset[randint(0, dataset_length)]
        # entry = dataset[count + 606]
        model_input, labels = entry
        skeleton = labels["skeleton"]
        skeleton_3d = labels["skeleton_3d"]
        scale_factor = 3
        skeleton_3d[:, :3] *= scale_factor
        skeleton_3d[:, :3] -= (scale_factor / 2)
        center = labels["center"]
        scale = labels["scale"]
        rotation = labels["rotation"]
        is_real_img = labels["is_real_img"]
        img_path = labels["img_path"]
        skeleton[:, 0] *= model_input.shape[1]
        skeleton[:, 1] *= model_input.shape[0]
        visible_joints = np.sum(skeleton[:, 2])
        body_orientation = labels["orientation"][0][1]
        body_orientation *= 2*math.pi
        body_orientation = math.degrees(body_orientation)
        head_orientation = labels["orientation"][1][1]
        head_orientation *= 2*math.pi
        head_orientation = math.degrees(head_orientation)
        draw_skeleton(model_input, skeleton)
        img = cv2.cvtColor(model_input, cv2.COLOR_BGR2RGB)
        ax[i, j].imshow(img)
        ax[i, j].set_title(f"{count}: {visible_joints} | Bθ​: {body_orientation:.1f}° | Hθ​: {head_orientation:.1f}°")

        ax_3d = fig_3d.add_subplot(3, 3, count+1, projection='3d')
        draw_grid_3d(ax_3d, lim=1)
        draw_origin_3d(ax_3d)
        add_skeleton_3d_to_axes(ax_3d, skeleton_3d, size=4)
        ax_3d.set_title(f"{count}: {visible_joints} | Bθ​: {body_orientation:.1f}° | Hθ​: {head_orientation:.1f}°")

        # print(f"{count}: Model input shape: {model_input.shape}, Min value: {model_input.min()}, max value: {model_input.max()}")
        # print(f"{count}: center: {center}, scale: {scale}, rotation: {rotation}")
        # print(f"{count}: is_real_img: {is_real_img}")
        # print(f"{count}: visible joints: {visible_joints}")
        # print(f"{count}: path: {img_path}")
        # print("------------")
        count += 1
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
dataset_df_path = os.path.join(dataset_root, dataset_df_filename)
df = pd.read_pickle(dataset_df_path)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")

Number of entrys: 147729


In [4]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
filter_skeleton3d = [col for col in df if col.startswith('skeleton3d')]
filter_bb = [col for col in df if col.startswith('bb')]
filter_body_orientation = [col for col in df if col.startswith('body_orientation')]
filter_head_orientation = [col for col in df if col.startswith('head_orientation')]
filter_env = [col for col in df if col.startswith('env')]

In [5]:
# skeleton 2d calculated columns
skeleton2d_xs = [col for col in df if col.startswith('skeleton2d') and col.endswith('_x')]
skeleton2d_ys = [col for col in df if col.startswith('skeleton2d') and col.endswith('_y')]
skeleton2d_visibles = [col for col in df if col.startswith('skeleton2d') and col.endswith('_visible')]
df["skeleton2d_width"] = df[skeleton2d_xs].max(axis=1) - df[skeleton2d_xs].min(axis=1)
df["skeleton2d_height"] = df[skeleton2d_ys].max(axis=1) - df[skeleton2d_ys].min(axis=1)
df["skeleton2d_size"] = np.sqrt(df["skeleton2d_width"]**2 + df["skeleton2d_height"]**2)
df["skeleton2d_visible_joints"] = df[skeleton2d_visibles].sum(axis=1)

In [6]:
# skeleton 3d calculated columns
skeleton3d_xs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_x')]
skeleton3d_ys = [col for col in df if col.startswith('skeleton3d') and col.endswith('_y')]
skeleton3d_zs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_z')]
df["skeleton3d_width"] = df[skeleton3d_xs].max(axis=1) - df[skeleton3d_xs].min(axis=1)
df["skeleton3d_height"] = df[skeleton3d_ys].max(axis=1) - df[skeleton3d_ys].min(axis=1)
df["skeleton3d_depth"] = df[skeleton3d_zs].max(axis=1) - df[skeleton3d_zs].min(axis=1)
df["skeleton3d_size"] = np.sqrt(df["skeleton3d_width"]**2 + df["skeleton3d_height"]**2 + df["skeleton3d_depth"]**2)

In [7]:
# distance calculated column
env_position_xs = [col for col in df if col == "env_position_x"] 
env_position_ys = [col for col in df if col == "env_position_y"] 
env_position_zs = [col for col in df if col == "env_position_z"] 

df["distance_xz"] = np.sqrt(np.abs(df[env_position_xs].sum(axis=1)**2) + np.abs(df[env_position_zs].sum(axis=1)**2))

df["body_orientation_phi"] = df["body_orientation_phi"] * math.pi * 2
df["body_orientation_theta"] = df["body_orientation_theta"] * math.pi
df["head_orientation_phi"] = df["head_orientation_phi"] * math.pi * 2
df["head_orientation_theta"] = df["head_orientation_theta"] * math.pi


In [9]:
df.head(5)

,dataset,dataset_type,scene_id,scene_start,scene_end,frame_nr_global,frame_nr_local,img_dir,img_id,img_type,subject_id,gender,skin_color,size,bmi,age,movement,movement_speed,is_real_img,actions,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,bb_class,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,body_orientation_visible,head_orientation_theta,head_orientation_phi,head_orientation_score,head_orientation_visible,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_nose_visible,skeleton2d_nose_supported,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_left_eye_visible,skeleton2d_left_eye_supported,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_right_eye_visible,skeleton2d_right_eye_supported,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_left_ear_visible,skeleton2d_left_ear_supported,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_right_ear_visible,skeleton2d_right_ear_supported,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_left_shoulder_visible,skeleton2d_left_shoulder_supported,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_right_shoulder_visible,skeleton2d_right_shoulder_supported,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_left_elbow_visible,skeleton2d_left_elbow_supported,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_right_elbow_visible,skeleton2d_right_elbow_supported,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_left_wrist_visible,skeleton2d_left_wrist_supported,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_right_wrist_visible,skeleton2d_right_wrist_supported,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_left_hip_visible,skeleton2d_left_hip_supported,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_right_hip_visible,skeleton2d_right_hip_supported,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_left_knee_visible,skeleton2d_left_knee_supported,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_right_knee_visible,skeleton2d_right_knee_supported,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_left_ankle_visible,skeleton2d_left_ankle_supported,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_right_ankle_visible,skeleton2d_right_ankle_supported,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_hip_center_visible,skeleton2d_hip_center_supported,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_spine_center_visible,skeleton2d_spine_center_supported,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_neck_visible,skeleton2d_neck_supported,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_lower_visible,skeleton2d_head_lower_supported,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_head_upper_visible,skeleton2d_head_upper_supported,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_left_foot_end_visible,skeleton2d_left_foot_end_supported,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_right_foot_end_visible,skeleton2d_right_foot_end_supported,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_left_hand_end_visible,skeleton2d_left_hand_end_supported,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d

In [10]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,frame_nr_global,frame_nr_local,img_id,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,head_orientation_theta,head_orientation_phi,head_orientation_score,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d_right_hand_end_score,skeleton3d_nose_x,skeleton3d_nose_y,skeleton3d_nose_z,skeleton3d_nose_score,skeleton3d_left_eye_x,skeleton3d_left_eye_y,skeleton3d_left_eye_z,skeleton3d_left_eye_score,skeleton3d_right_eye_x,skeleton3d_right_eye_y,skeleton3d_right_eye_z,skeleton3d_right_eye_score,skeleton3d_left_ear_x,skeleton3d_left_ear_y,skeleton3d_left_ear_z,skeleton3d_left_ear_score,skeleton3d_right_ear_x,skeleton3d_right_ear_y,skeleton3d_right_ear_z,skeleton3d_right_ear_score,skeleton3d_left_shoulder_x,skeleton3d_left_shoulder_y,skeleton3d_left_shoulder_z,skeleton3d_left_shoulder_score,skeleton3d_right_shoulder_x,skeleton3d_right_shoulder_y,skeleton3d_right_shoulder_z,skeleton3d_right_shoulder_score,skeleton3d_left_elbow_x,skeleton3d_left_elbow_y,skeleton3d_left_elbow_z,skeleton3d_left_elbow_score,skeleton3d_right_elbow_x,skeleton3d_right_elbow_y,skeleton3d_right_elbow_z,skeleton3d_right_elbow_score,skeleton3d_left_wrist_x,skeleton3d_left_wrist_y,skeleton3d_left_wrist_z,skeleton3d_left_wrist_score,skeleton3d_right_wrist_x,skeleton3d_right_wrist_y,skeleton3d_right_wrist_z,skeleton3d_right_wrist_score,skeleton3d_left_hip_x,skeleton3d_left_hip_y,skeleton3d_left_hip_z,skeleton3d_left_hip_score,skeleton3d_right_hip_x,skeleton3d_right_hip_y,skeleton3d_right_hip_z,skeleton3d_right_hip_score,skeleton3d_left_knee_x,skeleton3d_left_knee_y,skeleton3d_left_knee_z,skeleton3d_left_knee_score,skeleton3d_right_knee_x,skeleton3d_right_knee_y,skeleton3d_right_knee_z,skeleton3d_right_knee_score,skeleton3d_left_ankle_x,skeleton3d_left_ankle_y,skeleton3d_left_ankle_z,skeleton3d_left_ankle_score,skeleton3d_right_ankle_x,skeleton3d_right_ankle_y,skeleton3d_right_ankle_z,skeleton3d_right_ankle_score,skeleton3d_hip_center_x,skeleton3d_hip_center_y,skeleton3d_hip_cen

In [11]:
df_description = df.describe()
num_3d_models = 1
num_animations = 1
print("General Report")
print(f"Number of frames: {len(df)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")
print(f"Skeleton 3D Diameter & ${df_description['skeleton3d_size']['mean']:.2f}mm$ & ${df_description['skeleton3d_size']['std']:.2f}mm$ & ${df_description['skeleton3d_size']['min']:.2f}mm$ & ${df_description['skeleton3d_size']['max']:.2f}mm$ \\\\")
print(f"Camera Distance (XZ) & ${df_description['distance_xz']['mean']:.2f}mm$ & ${df_description['distance_xz']['std']:.2f}mm$ & ${df_description['distance_xz']['min']:.2f}mm$ & ${df_description['distance_xz']['max']:.2f}mm$ \\\\")

General Report
Number of frames: 147729
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $503.04px$ & $502.41px$ & $30.69px$ & $2202.49px$ \\
Skeleton 3D Diameter & $1816.89mm$ & $401.47mm$ & $169.13mm$ & $2840.49mm$ \\
Camera Distance (XZ) & $7204.94mm$ & $5578.22mm$ & $22.42mm$ & $21289.16mm$ \\


In [33]:
body_phi_deg = df['body_orientation_phi'] * (180 / math.pi)
max_val = 0
for i in range (0, 370, 10):
  elements = body_phi_deg[body_phi_deg.between(i, i+10, "both")]
  num_els = len(elements)
  max_val = max_val if max_val > num_els else num_els
  print(f"{i},{num_els}")


0,7220
10,5348
20,1677
30,714
40,577
50,602
60,785
70,1453
80,1513
90,1252
100,965
110,2201
120,1498
130,1285
140,2759
150,2183
160,1493
170,1105
180,1100
190,1154
200,1235
210,2290
220,1445
230,1552
240,3219
250,3855
260,5676
270,10952
280,14475
290,22230
300,7302
310,6493
320,6336
330,8157
340,7508
350,8120
360,0


In [12]:
fig = plt.figure()
body_phi_deg = df['body_orientation_phi'] * (180 / math.pi)
body_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/romb_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.7\\textwidth',
  'font=\\footnotesize',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig = plt.figure()
head_phi_deg = df['head_orientation_phi'] * (180 / math.pi)
head_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/romb_head_orientation_phi.tex", extra_axis_parameters={
  'width=0.7\\textwidth',
  'font=\\footnotesize',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig = plt.figure()
df['skeleton2d_width'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [15]:
fig = plt.figure()
df['skeleton2d_height'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [16]:
fig = plt.figure()
df['skeleton2d_size'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [17]:
fig = plt.figure()
df['skeleton3d_width'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [18]:
fig = plt.figure()
df['skeleton3d_height'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [19]:
fig = plt.figure()
df['skeleton3d_depth'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [20]:
fig = plt.figure()
df['skeleton3d_size'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [21]:
fig = plt.figure()
df['body_orientation_phi'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [22]:
fig = plt.figure()
df['head_orientation_theta'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [23]:
fig = plt.figure()
df['head_orientation_phi'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>